# MP-3 Data Analysis

> **Instructions**
>
>  Your Name:
>
>  Your Groupmate:
>
>  Other acknowledgements/resources used:
>
>
> There will be ✏️ checkpoints with blank cells for you to fill in. You are responsible for submitting an attempt for every checkpoint in this notebook.
>
>  You are encouraged to work on these in groups. However, **each student will write their own solutions and writeups and upload both to Canvas individually**.

## Getting Started

If you encounter any import errors, you may need to run the following command within this notebook:

```
!pip install psmpy statsmodels nltk seaborn
```

add any additional packages throwing you errors!

In [ ]:
# Install a package for propensity score matching
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime
import random
import statsmodels.formula.api as smf
from nltk.probability import FreqDist

sns.set_style("whitegrid")

First, we'll load the dataset. Each row is a single post made in r/aww in the year 2022. A few columns to note:

- `id`: The id of the post
- `author`: The author's display name
- `created_utc`: Timestamp of when the post was created
- `gilded`: The number of gold the post received
- `score`: The number of aggregated upvotes and downvotes the post received
- `title`: The original title of the post

We'll exclude all the comments by accounts since deleted to respect author privacy.

In [ ]:
df = pd.read_csv("r_aww_subs_2022.csv.xz",
                 usecols = ['id','author','created_utc',
                            'gilded', 'score', 'title']
                )

# Drop posts made by deleted accounts
df = df.query(f"author != '[deleted]'").reset_index(drop=True)
df.head(5)

### Adding Data

We're going to want some more information about our posts to help us find appropriate matches.

* `weeks_since_start`: which week the post was created in since the earliest post in this dataset.
* `title_length`: the length of the post title to give some metadata about the post content.
* `date`: the date of the post formatted nicely

In [ ]:
# The number of seconds in a week
ONE_WEEK = 60 * 60 * 24 * 7

df = df.assign(date=lambda df: pd.to_datetime(df.created_utc, unit="s"))

# Also keep track of when this post was made
df['weeks_since_start'] = (df.created_utc - df.created_utc.min()) // ONE_WEEK

# Length of the title in characters
df['title_length'] = df.title.apply(len)
df = df.drop(columns=['title'])

### ✏️ **Checkpoint 1 (1 point)**

There are two tasks for this checkpoint.

**Part 1:** Plot a histogram of the number of posts over time (i.e., using `date`). (0.5 points)

**Part 2:** In the writeup, state what you notice about the overall trends from the plot. (0.5 points)

In [ ]:
# Part 1: Your Code Here

**Part 2:** *Fill in this cell with your observations*

...

## Separating the Treatment and Control Authors

In order to answer our RQs using a causal inference setup, we need to identify a treatment group and a control group. We can then find appropriate matches between the groups to make causal claims about how receiving the treatment (i.e., high score) changes future user behavior.

With that in mind:
* Our **treatment** group: all users who have recieved high score on at least one post in the dataset
* Our **control** group: all users who have made at least one post in the dataset but not received high score on any of them. This is also referred to as the *placebo*.

We'll start by identifying what it means to receive "high score." We can do this in lots of ways, but for this MP we're going to pick the 2,000 highest scores to call "treated". A small treatment group like this makes sure the rest of this MP runs without too much delay!

After running the cell below, we'll have a boolean `high_score` column indicating whether the given post received high score.

In [ ]:
df = df.sort_values(by=['score', 'created_utc'], ascending=[False, True]).assign(high_score=[True]*2000 + [False]*(len(df) - 2000))
df[['score','high_score']]

### ✏️ **Checkpoint 2 (1 point)**

You have two tasks:

**Part 1:** Compile lists of unique authors in the treatment and control groups. (0.5 points)

**Part 2:** Create a new column in `df` called `condition` and assign each row a 0 if it corresponds to a post made by a control author, and a 1 for treated authors. *Note: this is different than the top 2,000 scores!* (0.5 points)

*Hint: you can use the `.loc` function to set values for a specific column based on a condition (see [Setting values](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html#setting-values)).*

In [ ]:
# Your code here
treated_authors = []
control_authors = []

##### Verify Your Solution

In [ ]:
assert len(treated_authors) == 1247, f'Number of treated authors not quite right ({len(treated_authors):,}).'
assert len(df.query('condition == 1')) == 13_972, f'Number of posts by treated authors not quite right ({len(df.query("treated == 1")):,}).'
assert all(df.query('condition == 1').groupby("author")['high_score'].sum() > 0), f'Some treated authors have no highly-scored posts!'

assert len(control_authors) == 105_569, f'Number of control authors not quite right ({len(control_authors):,}).'
assert len(df.query('condition == 0')) == 207_741, f'Number of posts by control authors not quite right ({len(df.query("treated == 0")):,}).'
assert all(df.query('condition == 0').groupby("author")['high_score'].sum() == 0), 'Some control authors have highly-scored posts!'

print('All tests passed!')

### Identifying Candidates for Matching

Now we've identified all the authors who received the treatment at some point and those that didn't. But let's say an author made multiple posts that got high score, do they all count as treatments?

For this MP, we'll make the simplifying assumption that **an author's first highly-scored post is their only treatment**. For the control set, we will select **a random placebo post from each control author**. We acknowledge, however, that our dataset is an arbitrary point in time, and their first gilded post within this specific window is not necessarily their first gilded post of all time.

Next we'll isolate those treatment and placebo posts into a dataframe of matching candidates, `df_candidates`.

In [ ]:
df_candidates = pd.concat([
    # Treatment candidates: a user's first highly-scored post
    df.query('high_score == 1 and condition == 1').sort_values(by='created_utc').drop_duplicates(subset=['author'], keep='first'),
    # Control candidates: a random placebo post from each control author
    df.query('high_score == 0 and condition == 0').sample(frac=1, random_state=10).drop_duplicates(subset=['author'])
    ])

## Propensity Score Matching

Now that we have our candidates, we have to figure out how to pair each treatment candidate with a *similar enough* control candidate. We can do this through **Propensity Score Matching (PSM)**, which uses logistic regression and a set of covariates to assign each candidate with a likelihood of being treated, regardless of whether it's a treatment or control sample.

For example, imagine someone posted a really high-quality post, but for whatever reason, it went under the radar and did not receive many upvotes. This is a perfect control sample because it probably *should* have been highly scored, but wasn't, so it may have a similar propensity score to an actual highly scored post. With high quality matches, when we look at how treated authors' behavior changes after receiving treatment, we can attribute those effects to the treatment itself.

For this lab, we're considering the following covariates:
* `baseline_num_posts`: the number of posts made by the author in the 2 weeks prior to treatment/placebo
* `baseline_avg_score`: the average score on all posts made by the author in the 2 weeks prior to treatment/placebo
* `baseline_avg_title_length`: the average length of post titles by the author
* `weeks_since_start`: which week this post was made

#### Calculating Covariates

Because our covariates focus on the 2 weeks of behavior prior to treatment/placebo, we'll need to find all the posts made by each author in that window.

We'll use the Pandas `merge` function to match up each candidate with *all* its author's other posts in the dataset. Then we can filter out all the posts that happened 2 weeks prior to treatment/placebo and take some aggregates!

In [ ]:
# Get pre- and post-treatment/placebo behavior for all candidate posts
df_agg = df_candidates.merge(df[['id','author', 'created_utc', 'score', 'title_length']],
                  on='author', suffixes=('', '_agg'), how='left'
                  ).query('id != id_agg')

# Define the week based on when each post ocurred relative to the author's other posts
df_agg['week'] = (df_agg['created_utc_agg'] - df_agg['created_utc']) // ONE_WEEK

# Aggregate covariate for matching in the two weeks prior to treatment/placebo
baseline = df_agg.query('-2 <= week < 0').groupby(["id"]).agg(
    baseline_num_posts=("created_utc", "count"),
    baseline_avg_score=("score_agg", "mean"),
    baseline_avg_title_length=("title_length_agg", "mean")
)

baseline.head(5)

Now let's add those covariates back into our candidate dataframe so we can use them in our matching!

In [ ]:
df_candidates = df_candidates.merge(baseline, on='id')
df_candidates.sample(5)

Let's actually perform matching now! This code will take our candidates and the relevant covariates and find each treatment candidate a single control match. This library is essentially computing a post's propensity of being treated by running a logistic regression model like this:
```
condition ~ baseline_num_posts + baseline_avg_score + weeks_since_start + baseline_avg_title_length
```
Then, we perform K-Nearest Neighbor (KNN) matching and drop any unmatched treatment or control candidates.

In [ ]:
from psmpy import PsmPy
from psmpy.functions import cohenD
from psmpy.plotting import *

In [ ]:
# Perform matching
psm = PsmPy(df_candidates[['id','condition', 'baseline_num_posts','baseline_avg_score','weeks_since_start', 'baseline_avg_title_length']],
            treatment='condition', indx='id', exclude = [])
psm.logistic_ps(balance = True)
psm.knn_matched(matcher='propensity_logit', replacement=False, caliper=None, drop_unmatched=True)

# Grab matched_IDs and propensity scores
df_matched = df_candidates.merge(psm.df_matched[['id', 'propensity_score','propensity_logit', 'matched_ID']],
                    on=['id'], how='inner')

df_matched.head(5)

### Measuring Match Quality

Now we have treatment-control pairs, but are they well-matched? We can measure the **Standardized Mean Difference (SMD)** of each of our covariates before and after matching to estimate match quality. Low SMDs indicate little difference between the treatment group and control group on a given covariate. Ideally, matching will decrease the SMD for each covariate and not exceed a certain threshold. For this MP, we'll consider **covariates to be well matched if they do not exceed an SMD of 0.25**.

The `PsmPy` library gives us a nice function to plot the SMDs, let's see if our matches look good!


In [ ]:
plt.figure(figsize=(8, 5))

# Plot SMDs by covariate
psm.effect_size_plot(before_color='#FCB754', after_color='#3EC8FB')

# Add line for threshold of match quality
plt.axvline(0.25, linestyle='--', color='red')

# # Adjust labels
plt.title('Match Quality')
plt.xlabel('Standardized Mean Difference (SMD)')
plt.ylabel('Covariate')
plt.show()

### ✏️ **Checkpoint 3 (2 points)**

Let's improve our match quality! Identify the covariate(s) with poor match quality and do the following:

**Part 1:** Plot a histogram (using `sns.histplot()`) of any poorly matched covariate(s) over the treatment and control candidates (i.e., pre-matched data, `df_candidates`). Do not plot any well-matched covariates. (0.5 points)

**Part 2:** Describe the shape of this distribution and state how we can adjust the shape to be more normally distributed. (0.5 points)

**Part 3:** Use the approach from part 2 to normalize the covariate's data. Then, re-match the data with the adjusted covariate and report match quality (i.e., plot SMDs as above). (1 points)

In [ ]:
# Part 1: Your Code Here

**Part 2: Fill in this cell**

...

## Wilcoxon Signed-Rank Test

Let's return to our RQs and see if the treated users' posting rate and average score changes significantly more after treatment compared to their control matches.

To do this, we'll employ a statistical test called the **Wilcoxon signed-rank test**, which allows us to determine if the difference between outcomes for paired samples (e.g., our matches) is stastistically signifcant.

The null hypothesis for our test is as follows:
> **The median of the paired differences is zero**

In our context, that means treated users' posting rate and average score change by the same amount as the control users'.

To carry out this test, we first need to grab each matched user's posts before and after their treatment/placebo so we can examine how their behavior changed.

In [ ]:
# Get pre- and post-treatment/placebo behavior for all candidate posts
df_pre_post = df_matched.merge(df[['id','author', 'high_score', 'created_utc', 'score']],
                  on='author', suffixes=('', '_agg')
                  ).query('id != id_agg')

# Add a boolean for whether the post was pre- or post-treatment
df_pre_post['post_treatment'] = df_pre_post['created_utc_agg'] >= df_pre_post['created_utc']
df_pre_post.head(5)

Now we want to compute a delta for each user and each outcome. Essentially, how much did their `num_posts` and `avg_score` change after treatment/placebo? We can use the pandas [`pivot_table()`](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#pivot-and-pivot-table) function to make a table with a row for each post, and columns for each combination of outcome and `post_treatment`.

In [ ]:
# Compare pre and post treatment values for each outcome
df_pre_post = df_pre_post.groupby(['id','author','post_treatment']).agg(
    avg_score=('score_agg','mean'),
    num_posts=('created_utc','count'),
).reset_index()

# Pivot the table based on 'post_treatment' (0 and 1)
df_pre_post = df_pre_post.pivot_table(index=['id', 'author'], columns='post_treatment', values=['avg_score', 'num_posts'])

# Rename the columns
df_pre_post.columns = [f'{col[0]}_{"post" if col[1] else "pre"}' for col in df_pre_post.columns]

df_pre_post.head(5)

Now let's compute those deltas! After this cell, we'll have the following new columns:
* `avg_score_delta`: the difference between an author's average score before and after treatment/placebo
* `num_posts_delta`: the difference between an author's number of posts before and after treatment/placebo

In [ ]:
# Fill in 0s if there is no posting data for an author
df_pre_post[['num_posts_pre','num_posts_post']] = df_pre_post[['num_posts_pre','num_posts_post']].fillna(0)

# Compute the differences (post-treatment minus pre-treatment)
df_pre_post['avg_score_delta'] = df_pre_post['avg_score_post'] - df_pre_post['avg_score_pre']
df_pre_post['num_posts_delta'] = df_pre_post['num_posts_post'] - df_pre_post['num_posts_pre']

df_pre_post = df_pre_post[['avg_score_delta','num_posts_delta']]

df_pre_post.head(5)

Next, our Wilcoxon signed-rank test is paired, meaning we want to compare the delta for each user with the delta for their matched user. Let's modify the dataframe to have one row per pair. Columns with a `_t` suffix correspond to the treatment author and those with a `_c` suffix correspond to the control match.

In [ ]:
# Bring back condition information
df_pre_post = df_pre_post.merge(df_matched[['id','condition','matched_ID']], on=['id'])

# Get control and treatment matches in the same row
df_pre_post = df_pre_post.query('condition == 1').merge(df_pre_post.query('condition == 0'),
                                           left_on='matched_ID',
                                           right_on='id',
                                           how='inner',
                                           suffixes=('_t','_c'))
df_pre_post.head(5)

### ✏️ **Checkpoint 4 (1 point)**

Conduct Wilxocon signed-rank tests to answer the following: 

**Part 1:** Did the treatment group's `avg_score` and `num_posts` change significantly more or less after treatment than the control group's `avg_score` and `num_posts` after placebo? (0.5 points)

**Part 2:** State which group has larger changes resulting from treatment/placebo and whether it is statistically significant using the significance level $\alpha = 0.05$. (0.5 point)


Utilize the `wilcoxon` function from [`scipy.stats`](https://docs.scipy.org/doc//scipy-1.10.1/reference/generated/scipy.stats.wilcoxon.html), imported below.

*Hint: to determine which group, treatment or control, has a larger difference, look at the `alternative` parameter to the `wilcoxon` function.*

In [ ]:
from scipy.stats import wilcoxon

# Your code here

**Part 2: Fill in this cell**

...

## Set up regression analysis

To set up our dataset for the upcoming regression analysis, we'll need to do some organization of the data. The regression will take into account the treatment and control comments themselves, but also the pre- and post- treatment/placebo activity of the users, similar to the Wilcoxon signed-rank test, but with more granular time steps.

First, let's merge our `matched_ID`s and other metadata about the matches back into the original dataframe.



In [ ]:
df = df.merge(df_matched[['id','author','matched_ID','created_utc', 
                          'baseline_num_posts','baseline_avg_score','weeks_since_start', 'baseline_avg_title_length']
                        ], on='author',suffixes=('', '_treatment'), how='inner')
df.head(5)

### ✏️ **Checkpoint 5 (1 point)**

Define a new column in `df` called `weeks_since_treatment` that stores an **integer** value representing how many weeks since an author's treatment/placebo time a post was made. This column represents relative time with respect to treatment/placebo, as opposed to absolute time (i.e., Jan 3, 2022).

*Hint: each row is a post in the dataset which may or may not have been a treatment/control candidate. The column `created_utc_treatment` corresponds to the time the post's author received the treatment or placebo.*

In [ ]:
# Your code here
...

##### Verify Your Solution

In [ ]:
# Test Cases
assert df.query('id == "yyzghh"').weeks_since_treatment.values[0] == 44 and \
       df.query('id == "t2ubfp"').weeks_since_treatment.values[0] == -4 and \
       df.query('id == "txhmmc"').weeks_since_treatment.values[0] == 0, 'Something is not quite right. Double check your "weeks_since_treatment" calculation.'

print('All tests passed!')

## Aggregating Data

In the Wilcoxon signed-rank test, we aggregated all of an author's posting behavior before treatment/placebo and after. For the regression, we want a more granular analysis, so we're going to create weekly groupings so we can see how an author's behavior changes over time.

### ✏️ **Checkpoint 6 (1 point)**

For this checkpoint, we're going to compute weekly statistics for each treatment or control sample.

**Part 1:** Identify the rows of `df` corresponding to posts that did *NOT* receive the treatment or placebo. For this checkpoint, we only want to look at the posts made by treatment/control authors before or after treatment/placebo. Use [`pd.groupby`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) to group the result into weekly groups (using `weeks_since_treatment`) based on each treatment/control sample (indicated by `id_treatment`). *Hint: you can group by multiple columns!* (0.5 points)

**Part 2:** For each sample-week pair, calculate the average weekly score (`avg_score`) and the average number of posts (`num_posts`). *Hint: use the [`agg()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html) function!* (0.5 points)

Save these weekly aggregates in a dataframe called `df_weekly`.

In [ ]:
# Your code here
df_weekly = ...

##### Verify Your Solution

In [ ]:
# Test cases
assert len(df_weekly) == 4_962, 'Unexpected number of weekly aggregates. Check your groupby'
assert df_weekly.query('id_treatment == "sllppw" and weeks_since_treatment == 40').num_posts.values[0] == 2, 'Values in num_posts are not quite right.'
assert df_weekly.query('id_treatment == "t0entv" and weeks_since_treatment == -2').avg_score.values[0] == 169, 'Values in avg_score are not quite right.'
assert len(df_weekly.columns.union(set(['id_treatment', 'weeks_since_treatment', 'avg_score','num_posts']))) == 4, 'Columns in df_weekly are not quite right.'
print('All tests passed!')

Now that you've constructed the weekly aggregates, we'll add in the author and condition information so we have it for the regression.

We'll also define a boolean `post_treatment` to indicate whether a post occurred before or after treatment/placebo time.

In [ ]:
df_weekly = df_weekly.merge(df[['id_treatment','author', 'condition',
                                'baseline_num_posts', 'baseline_avg_score', 'weeks_since_start', 'baseline_avg_title_length'
                               ]].drop_duplicates(), on='id_treatment', how='left')

df_weekly['post_treatment'] = df_weekly['weeks_since_treatment'] >= 0

df_weekly.sample(10)

To help visualize this dataframe, we can plot one author's posting behavior in our dataset. You'll see that they have posts in `df_weekly` from before their treatment (i.e., `weeks_since_treatment = 0`) and after.

In [ ]:
sns.set_style("whitegrid")

sns.lineplot(data = df_weekly.query('author == "child-of-old-gods"'), x='weeks_since_treatment', y='num_posts')
plt.axvline(0, color='red', linestyle='--')
plt.title('Posts by author "child-of-old-gods"')
plt.show()

## Difference-in-Differences Analysis

As a reminder, we have two RQs we're hoping to answer:

> **RQ 1: How does being highly upvoted on a post affect an author's posting frequency?**
>
> **RQ 2: How does being highly upvoted on a post affect an author's future score?**

We can answer these questions using a **Difference-in-Differences (DiD)** approach, a technique that allows us to use the *counterfactual* (i.e., what would have happened to the treatment group if they hadn't been treated) to estimate causal effects. Here's a [helpful source](https://www.publichealth.columbia.edu/research/population-health-methods/difference-difference-estimation) on DiD regression.

DiD analysis compares matched control users before/after placebo against what happened to the treated users after treatment. It's impossible to simultaneously apply the treatment and placebo to a single user, so we use DiD analysis to estimate that coutnerfactual using interaction terms. Here's a [helpful diagram](https://www.publichealth.columbia.edu/research/population-health-methods/difference-difference-estimation) to visualize how this works!

The general form you should follow is:

```
outcome ~ post_treatment * condition + covariate_1 + covariate_2 + ...
```

###  ✏️️ **Checkpoint 7 (3 points)**

For this checkpoint, you use a DiD regression setup to model the impact of the treatment on the outcomes (`avg_score` and `num_posts`), while controlling for the general trend in posting activity over time as well as differences between the treatment and control group.

We will also incorporate the matching information into the analysis. One way would be to add a unique ID for each treatment-control pair, but that slows the model down significantly and adds a lot of hard-to-parse coefficients. Instead, **add the matching covariates as independent variables to the regression formula**. If you need a reminder, here are the covariates we matched on:
* `baseline_num_posts`
* `baseline_avg_score`
* `weeks_since_start`
* `baseline_avg_title_length`


Your task is to:

**Part 1.** Since `avg_score` is a continuous variable, use [`smf.ols`](https://www.statsmodels.org/devel/generated/statsmodels.formula.api.ols.html) to fit an ordinary least squares model to the data for the `avg_score` outcome. (1 point)

**Part 2.** Since `num_posts` is count data, use [`smf.negativebinomial`](https://www.statsmodels.org/devel/generated/statsmodels.formula.api.negativebinomial.html) to fit a negative binomial model to the data for the `num_posts` outcome. *Note: Poisson regression can also be used on count data, but our data exhibits [overdispersion](https://online.stat.psu.edu/stat504/lesson/7/7.3) which is handled better by Negative Binomial models.* (1 point)

Note: You may see a bunch of `RuntimeWarning`s. You can safely ignore them.

**Part 3.** Interpret the results, answering the following questions (with numbers). We're giving you a template solution since the interpretations can be tricky! Replace all the $\beta$s with real coefficients, fill in the planks, and select one of the bolded words, where applicable (1 point)

- How much higher is treated users' average score in comparison to control users, before any treatment? Is this result statistically significant?
- How much do treated users post after treatment compared to before? What about control users?
- How much higher are treated users' average score *and* number of posts compared to what they would have been if they had not received the treatment (i.e., the counterfactual)?

*Hint: Negative Binomial models have an exponential link function, so the coefficients represent relative change. To interpret them more easily, you will have to exponentiate (i.e., `np.exp`) the coefficients and interpret the resulting number as a percent.*

In [ ]:
# Part 1: Your Code Here

In [ ]:
# Part 2: Your Code Here

**Part 3: Fill in this cell**

From the OLS model:
* Treated users' average score is $\beta_\texttt{condition}$ upvotes **higher/lower** than control users' score, before treatment was applied.
* Treated users' average score is $\beta_\texttt{post_treatment:condition}$ upvotes **higher/lower** than it would have been had they not received the treatment. 

From the Negative Binomial Model:
* Treated users post $(\exp(\beta_\texttt{post_treatment}+\beta_\texttt{post_treatment:condition}) - 1)*100 \approx$ \_\_\_% **more/less** after treatment. 
* Control users post $(\exp(\beta_\texttt{post_treatment}) - 1)*100 \approx$ \_\_\_% **more/less** after treatment.
* Treated users' number of posts is $(\exp(\beta_\texttt{post_treatment:condition}) - 1)*100 \approx$ \_\_\_% **higher/lower** than it would have been had they not received the treatment.
